In [73]:
# Pkg.add("DataFrames")
# Pkg.add("Requests")
# Pkg.add("MIToS")
using DataFrames
using Requests
using MIToS.SIFTS

function getCSA()
    url = "http://www.ebi.ac.uk/thornton-srv/databases/CSA/downloads/CSA_2_0_121113.txt"
    csa_file = "csa.csv"
    
    if (stat(csa_file).mode == 0) 
        stream = Requests.get_streaming(url)
        headers(stream)
        response = readall(stream) 
        open(csa_file, "w") do file
            write(file, response)
        end        
    end
    readtable(csa_file)
end

function getUniProtChainResidues(pdb_accessor, the_chain, the_number)
    siftsfile = string(pdb_accessor,".xml.gz")
    if (stat(siftsfile).mode == 0)
        siftsfile = downloadsifts(pdb_accessor)
    end
    siftsresidues = read(siftsfile, SIFTSXML, chain=the_chain, missings=false) # Array{SIFTSResidue,1}

    indexes = findobjects(siftsresidues, dbPDB, SIFTS.In(:number, the_number))

    [ get(siftsresidues[ idx ].UniProt) for idx in indexes ] # UniProt data of the basic residues
end


getUniProtChainResidues (generic function with 1 method)

In [10]:
catalytic_sites = getCSA()
head(catalytic_sites)

,PDB_ID,SITE_NUMBER,RESIDUE_TYPE,CHAIN_ID,RESIDUE_NUMBER,CHEMICAL_FUNCTION,EVIDENCE_TYPE,LITERATURE_ENTRY
1,12as,0,Arg,A,100,S,LIT,12as
2,12as,0,Asp,A,46,S,LIT,12as
3,12as,0,Gln,A,116,S,LIT,12as
4,132l,0,Asp,A,52,S,LIT,132l
5,132l,0,Glu,A,35,S,LIT,132l
6,135l,0,Asp,A,52,S,LIT,135l


In [84]:
# obtengo las que tiene evidencia en la literatura
literature_catalytic_sites= catalytic_sites[catalytic_sites[:EVIDENCE_TYPE] .== "LIT",:]
nrows(literature_catalytic_sites)

LoadError: LoadError: UndefVarError: nrows not defined
while loading In[84], in expression starting on line 3

In [61]:
getUniProtChainResidues("12as","A",[46,100,116])

0-element Array{Any,1}

In [74]:
getUniProtChainResidues("132l","A",[52,35])

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8976  100  8976    0     0   1284      0  0:00:06  0:00:06 --:--:--  2089


2-element Array{Any,1}:
 MIToS.SIFTS.dbUniProt("P00698",53,"E")
 MIToS.SIFTS.dbUniProt("P00698",70,"D")